<a href="https://colab.research.google.com/github/dominiquedeveraux/Used-Car-Prediction-Gradient-Boosting-Regression-Model/blob/main/Used_Car_Price_Prediction_Gradient_Boosting_Regres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'kagglex-cohort4:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5131699%2F8580803%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240601%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240601T213924Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4fccf2ec334ec2e00b08a61dffb80385fae5d172f3cb658db393d68c6966214f93ff69d302c96615a3a0a3e8ce6c613bbfad9aa2e4fee083e6c9a7132568a56e7189b95d5ef4b9b789880d8fc177eb2b7183f90d3b3e2d92beb7d48ae6c5b212bafb9cab5e1374642495360e61a0df6c66cec39dd9fc5dff30fba9f5aee0c1436c05cc7696f47e9cea2f487dc0f13e067e5e192c71f192e37b5f1740dd1b58346a3fffc552a92aa2f94f52091397a07b146ebae3b1c61f2843959ef74dfb8eb95d069a046e063bc17dbfdbb4fd0c2d807919208cee540308f11b348e2d2727195a563a75b0055bd28d0297fe72ac72386fb95f6f578d8d7129d9ba62b800b173'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/kagglex-cohort4/kagglex-cohort4'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

train_df = pd.read_csv('/kaggle/input/kagglex-cohort4/kagglex-cohort4/train.csv')
test_df = pd.read_csv('/kaggle/input/kagglex-cohort4/kagglex-cohort4/test.csv')

print(train_df.head())
print(test_df.head())


In [ ]:
import pandas as pd

# Reload the dataframes
train_df = pd.read_csv('/kaggle/input/kagglex-cohort4/kagglex-cohort4/train.csv')
test_df = pd.read_csv('/kaggle/input/kagglex-cohort4/kagglex-cohort4/test.csv')

# Print the column names to verify
print(train_df.columns)
print(test_df.columns)

# Perform one-hot encoding
train_df = pd.get_dummies(train_df, columns=['ext_col', 'int_col', 'accident', 'clean_title'])
test_df = pd.get_dummies(test_df, columns=['ext_col', 'int_col', 'accident', 'clean_title'])

print(train_df.head(2))
print(test_df.head(2))

In [ ]:
# Iterate through columns in train_df
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        train_df[col] = pd.to_numeric(train_df[col], errors='coerce')

# Iterate through columns in test_df
for col in test_df.columns:
    if test_df[col].dtype == 'object':
        test_df[col] = pd.to_numeric(test_df[col], errors='coerce')


In [ ]:
print('Train shape before removing NaN:', train_df.shape)
train_df = train_df.dropna()
print('Train shape after removing NaN:', train_df.shape)

print('Test shape before removing NaN:', test_df.shape)
test_df = test_df.dropna()
print('Test shape after removing NaN:', test_df.shape)


In [ ]:
# Check the number of NaN values in each column of train_df
print(train_df.isna().sum())

# Check the number of NaN values in each column of test_df
print(test_df.isna().sum())

In [ ]:
# Fill NaN values in numerical columns with the mean
train_df = train_df.fillna(train_df.mean(numeric_only=True))

# Fill NaN values in numerical columns with the mean
test_df = test_df.fillna(test_df.mean(numeric_only=True))

print('Train shape after imputing NaN:', train_df.shape)
print('Test shape after imputing NaN:', test_df.shape)


In [ ]:
import pandas as pd

# Reload the data
train_df = pd.read_csv('/kaggle/input/kagglex-cohort4/kagglex-cohort4/train.csv')
test_df = pd.read_csv('/kaggle/input/kagglex-cohort4/kagglex-cohort4/test.csv')

# Perform one-hot encoding again
train_df = pd.get_dummies(train_df, columns=['ext_col', 'int_col', 'accident', 'clean_title'])
test_df = pd.get_dummies(test_df, columns=['ext_col', 'int_col', 'accident', 'clean_title'])

# Remove string values from all columns
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        train_df[col] = pd.to_numeric(train_df[col], errors='coerce')

for col in test_df.columns:
    if test_df[col].dtype == 'object':
        test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

# Fill NaN values in numerical columns with the mean
train_df = train_df.fillna(train_df.mean(numeric_only=True))
test_df = test_df.fillna(test_df.mean(numeric_only=True))

print('Train shape after imputing NaN:', train_df.shape)
print('Test shape after imputing NaN:', test_df.shape)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate the model
model = GradientBoostingRegressor()

# Separate features and target variable
X = train_df.drop('price', axis=1)
y = train_df['price']

# Impute NaN values with mean for numerical columns
X = X.fillna(X.mean(numeric_only=True))

# Impute NaN values with 0 for one-hot encoded columns
X = X.fillna(0)

# Drop original non-numerical columns
X = X.select_dtypes(exclude=['object'])

# Train the model
model.fit(X, y)

In [ ]:
# Create a DataFrame with 'id' and 'price' columns
result_df = pd.DataFrame({'id': test_df['id'], 'price': predicted_prices})



In [ ]:
result_df = result_df.sort_values('price', ascending=True)
print(result_df)

